# May 10, 2024: visualize modes in 3D

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 

from scipy import sparse, stats
import glob
from tqdm import tqdm
import ants
from nipype.interfaces import afni
from itertools import product, combinations

import graph_tool.all as gt

from brainrender import Scene, actor

import vedo
vedo.settings.default_backend = 'k3d'
from vedo import Volume, Plotter

from brainglobe_utils.IO.image.load import load_nii

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT

from matplotlib import colors
from palettable.tableau import Tableau_20
import colorcet as cc


# # ignore user warnings
# import warnings
# warnings.filterwarnings("ignore") #, category=UserWarning)

240602-21:12:42,355 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.1


In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

gt.seed_rng(args.SEED)
np.random.seed(args.SEED)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

args.unit = 'seswise' # runwise/seswise/subwise : unit sample of dataset
args.denst = 20 # density of fc matrix

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results'
ROI_RESULTS_path = f'{ROI_path}/{DESC}/{args.unit}/density-{args.denst}'
FC_path = f'{ROI_RESULTS_path}/corr_mats'
SBM_path = f'{ROI_RESULTS_path}/sbms'
NPY_path = f'{ROI_RESULTS_path}/npy'
os.system(f'mkdir -p {NPY_path}')
NII_path = f'{ROI_RESULTS_path}/niis'
os.system(f'mkdir -p {NII_path}/indiv')
os.system(f'mkdir -p {NII_path}/group')
HTML_path = f'{ROI_RESULTS_path}/htmls'
os.system(f'mkdir -p {HTML_path}/group')
# sbm model class
args.dc = 'dc'
args.sbm = 'h' 

In [4]:
parcels_img = ants.image_read(f'{PARCELS_path}/{DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{DESC}_desc-labels.txt')

---

In [5]:
with open(f'{NPY_path}/sbm-{args.dc}-{args.sbm}_desc-group-modes.npy', 'rb') as f:
    [dfs, mode_df] = pickle.load(f)

In [6]:
mode_df

,mode_id,mode,omega,sigma,ratio,b
0,0,<graph_tool.inference.partition_modes.Partitio...,1.0,0.489546,2.042709,"[[0, 1, 1, 2, 3, 4, 5, 4, 0, 5, 4, 1, 0, 4, 4,..."


In [7]:
def add_vol(args, scene, vol, color='#cccccc', alpha=0.2, label='template'):
    mesh = vol.isosurface()
    mesh = mesh.smooth(
        niter=1000, 
        pass_band=0.1, 
        edge_angle=15, 
        feature_angle=150,
        boundary=False,
    )
    mesh_actor = actor.Actor(
        mesh, 
        name=f'{label}', 
        br_class='Volume', 
        color=color, 
        alpha=alpha
    )
    scene.add(mesh_actor)
    # scene.add_label(actor=mesh_actor, label=label)
    return scene

def visualize_nested_partition(args, template, data, colors_list, filename):
    def visualize_data(args, scene, vol, colors_list):
        comms = np.unique(vol)[1:].astype(int)
        
        for idx_comm, comm in enumerate(comms):
            color = colors_list[comm]
            print(comm, color)
            comm = (vol == comm)
            comm = Volume(comm, origin=[0,0,0], spacing=spacing)
            scene = add_vol(
                args, scene, vol=comm, 
                color=color,#colors_list[idx_comm], 
                alpha=alpha, 
                label=f'comm {idx_comm}'
            )
        return scene
    
    def change_filename(filename, level):
        fs = filename.split('/')
        fi = fs[-1]
        fis = fi.split('_')
        return '/'.join(fs[:-1] + ['_'.join(fis[:2] + [f'level-{level}'] + fis[2:])])
    
    template = load_nii(template, as_array=True)
    spacing=[args.spacing]*3
    vol_template = Volume(template, origin=[-75,0,0], spacing=spacing)
    
    data = load_nii(data, as_array=True)
    for level in range(data.shape[-1]):
        scene = Scene()
        root_brain = scene.get_actors()[0].alpha(0.0)

        scene = add_vol(
            args, scene, vol=vol_template, 
            color='#cccccc', alpha=0.2,
            label='template'
        )
        
        vol = data[:, :, :, level]
        spacing = [args.spacing]*3
        alpha = 1.0
        scene = visualize_data(args, scene, vol=vol, colors_list=colors_list)

        
        # plt = Plotter()
        # plt.show(*scene.renderables, zoom=True)
        scene.export(f'{change_filename(filename, level)}')
    return None

def visualize_partition(args, template, data, colors_list, filename):
    scene = Scene()
    root_brain = scene.get_actors()[0].alpha(0.0)

    template = load_nii(template, as_array=True)
    spacing=[args.spacing]*3
    vol = Volume(template, origin=[-75,0,0], spacing=spacing)
    scene = add_vol(
        args, scene, vol=vol, 
        color='#cccccc', alpha=0.2,
        label='template'
    )
    
    def visualize_data(args, scene, vol, colors_list):
        comms = np.unique(vol)[1:].astype(int)
        
        for idx_comm, comm in enumerate(comms):
            color = colors_list[comm % len(colors_list)]
            print(comm, color)
            comm = (vol == comm)
            comm = Volume(comm, origin=[0,0,0], spacing=spacing)
            scene = add_vol(
                args, scene, vol=comm, 
                color=color,#colors_list[idx_comm], 
                alpha=alpha, 
                label=f'comm {idx_comm}'
            )
        return scene
    
    data = load_nii(data, as_array=True)
    vol = data
    spacing = [args.spacing]*3
    alpha = 1.0
    scene = visualize_data(args, scene, vol=vol, colors_list=colors_list)
    
    # plt = Plotter()
    # plt.show(*scene.renderables, zoom=True)
    scene.export(f'{filename}')
    return None

In [8]:
# perceptually uniform custom colormap
import colorcet as cc
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors
def get_uniform_colors():
    colors_val = []
    for idx in np.arange(5, 56, 10):
        colors_val += cc.CET_C6s[idx:256:55]
    colors_val.pop(6)
    colors_rgb = [colors.to_rgb(c) for c in colors_val]
    return colors_val, colors_rgb

colors_val, colors_rgb = get_uniform_colors()
# bs = np.stack(mode_df['b'].to_list()).T
# max_num_colors = np.max(bs)+1
# colors_val = colors_val[:max_num_colors]
# colors_rgb = colors_rgb[:max_num_colors]

args.spacing = 200 #um
# comms = np.arange(1, 21)
colors_list = ['#cccccc'] + colors_val
colors_list


[
    '#cccccc',
    '#c3acff',
    '#f83827',
    '#d4ce00',
    '#31bf82',
    '#2791ff',
    '#eabdfb',
    '#a7bd00',
    '#2fd8b5',
    '#4280ff',
    '#ffb6d9',
    '#fc7401',
    '#77aa02',
    '#25e8e3',
    '#7989ff',
    '#ff97a3',
    '#ff9c00',
    '#479a15',
    '#2adcfb',
    '#aa9eff',
    '#ff706d',
    '#fec000',
    '#2c9d39',
    '#31beff',
    '#fc473c',
    '#e8d100',
    '#2fb269',
    '#2b9fff'
]

In [9]:
SBM = f'sbm-{args.dc}-{args.sbm}'
files = sorted(glob.glob(f'{NII_path}/group/*{SBM}*', recursive=True))
template = f'{BASE_path}/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'

def make_filename(file):
    fs = file.split('/')
    filename = fs[-1].split('_')
    filename = '_'.join(filename[:-1] + ['.'.join(filename[-1].split('.')[:1] + ['html'])])
    filename = '/'.join([f'{HTML_path}/group'] + [filename])
    return filename
    
for file in tqdm(files):
    data = file
    filename = make_filename(file)
    
    if args.sbm in ['a', 'd']:
        visualize_partition(args, template, data, colors_list, filename)
    if args.sbm in ['h']:
        visualize_nested_partition(args, template, data, colors_list, filename)

  0%|          | 0/1 [00:00<?, ?it/s]

1 #c3acff
2 #f83827
3 #d4ce00
4 #31bf82
5 #2791ff
6 #eabdfb
7 #a7bd00
8 #2fd8b5
9 #4280ff
10 #ffb6d9
11 #fc7401
12 #77aa02
13 #25e8e3
14 #7989ff
15 #ff97a3
16 #ff9c00


Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/mouse_dataset/roi_results/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/seswise/density-20/
htmls/group/sbm-dc-h_mode-0_level-0_omega-1.00_sigma-0.49_ratio-2.043_desc-partition.html

...

1 #c3acff


Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/mouse_dataset/roi_results/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/seswise/density-20/
htmls/group/sbm-dc-h_mode-0_level-2_omega-1.00_sigma-0.49_ratio-2.043_desc-partition.html

100%|██████████| 1/1 [00:13<00:00, 13.74s/it]
